# Efficient Yelp Api Calls

In [14]:
#Full Name: Hannah Ploutz

## Imports

In [57]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## Function

In [58]:
def create_json_file(JSON_FILE, delete_if_exists=False):

    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists == True:

        # Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # Delete file and confirm it no longer exists
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")

    # If it does NOT exist:
    else:

        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder:
        if len(folder) > 0:
            # Create the folder
            os.makedirs(folder, exist_ok=True)
        # Save empty list to start the JSON file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

## Load API Credentials

In [59]:
# Load API Credentials
with open('/Users/hgplo/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [60]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

In [61]:
#running a test query
test_results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(test_results))
test_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [62]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Salad'

### Use Function to create JSON File

In [63]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)

## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_salad.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_salad.json not found. Saving empty list to new file.
- 0 previous results found.


1015

### Use For Loop to Check if Data file exists

- if it doesnt, create folder and save empty list as JSON file
- then run the search and save the results to the file

In [64]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/1015 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


### Load Final Results + Cleanup

In [65]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,aMG1RR8UPLL4Kl33SQr9RA,chop-and-go-new-york-3,Chop & Go,https://s3-media2.fl.yelpcdn.com/bphoto/uR2IFp...,False,https://www.yelp.com/biz/chop-and-go-new-york-...,16,"[{'alias': 'salad', 'title': 'Salad'}]",4.5,"{'latitude': 40.759297, 'longitude': -73.970186}","[pickup, delivery]","{'address1': '643 Lexington Ave', 'address2': ...",+19173009546,(917) 300-9546,5866.859902,NaN
1,q2Vq56jenOx8cXD2fCupYg,leitao-new-york-3,Leitao,https://s3-media3.fl.yelpcdn.com/bphoto/AiCqtq...,False,https://www.yelp.com/biz/leitao-new-york-3?adj...,122,"[{'alias': 'portuguese', 'title': 'Portuguese'...",4.5,"{'latitude': 40.73495, 'longitude': -74.00628}","[pickup, delivery]","{'address1': '547 Hudson St', 'address2': '', ...",+12128109944,(212) 810-9944,3861.581136,$$$
2,vZevXSC1w27dEvXc2EHCEg,good-thanks-cafe-new-york,Good Thanks Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/8B_AcK...,False,https://www.yelp.com/biz/good-thanks-cafe-new-...,378,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 40.71972, 'longitude': -73.98952}","[pickup, delivery]","{'address1': '131A Orchard St', 'address2': No...",+16463704426,(646) 370-4426,1664.931111,$$
3,UK8Ju3V32B5g_kh2GNNMjA,just-salad-brooklyn-6,Just Salad,https://s3-media4.fl.yelpcdn.com/bphoto/_79nHL...,False,https://www.yelp.com/biz/just-salad-brooklyn-6...,44,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,"{'latitude': 40.7153455, 'longitude': -73.9584...","[pickup, delivery]","{'address1': '182 North 4th', 'address2': None...",+18666733757,(866) 673-3757,1983.652484,NaN
4,dqfHeckcMJLhrRSjlPIynw,avocaderia-brooklyn,Avocaderia,https://s3-media1.fl.yelpcdn.com/bphoto/rUVV-0...,False,https://www.yelp.com/biz/avocaderia-brooklyn?a...,245,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.5,"{'latitude': 40.656252, 'longitude': -74.007841}","[pickup, delivery]","{'address1': '238 36th St', 'address2': '', 'a...",+13472270350,(347) 227-0350,6136.171847,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
995,O9LKN5kk17lQWb7mMuiBzw,yasouvlaki-new-york,yasouvlaki,https://s3-media4.fl.yelpcdn.com/bphoto/w_Aciv...,False,https://www.yelp.com/biz/yasouvlaki-new-york?a...,109,"[{'alias': 'greek', 'title': 'Greek'}]",4.0,"{'latitude': 40.7803, 'longitude': -73.95312}",[],"{'address1': '1568 3rd Ave', 'address2': '', '...",,,8444.227049,NaN
996,jTvExXa9M8QqS_Wn_4KB5g,harissa-grill-long-island-city,Harissa Grill,https://s3-media4.fl.yelpcdn.com/bphoto/-cRhLz...,False,https://www.yelp.com/biz/harissa-grill-long-is...,16,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.0,"{'latitude': 40.7423867, 'longitude': -73.9364...","[pickup, delivery]","{'address1': '30-30 47th Ave', 'address2': '',...",+13479249090,(347) 924-9090,5336.312095,NaN
997,TQzGf4k3HXyMbnNkYKKQSw,oxalis-brooklyn,Oxalis,https://s3-media3.fl.yelpcdn.com/bphoto/6Yw9Aw...,False,https://www.yelp.com/biz/oxalis-brooklyn?adjus...,240,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 40.673109, 'longitude': -73.9626279}","[pickup, delivery]","{'address1': '791 Washington Ave', 'address2':...",+13476278298,(347) 627-8298,4016.488596,$$$$
998,VKihkof15rf3M4C-kuDL5Q,southern-charm-new-york,Southern Charm,https://s3-media4.fl.yelpcdn.com/bphoto/skLWt0...,False,https://www.yelp.com/biz/southern-charm-new-yo...,5,"[{'alias': 'tradamerican', 'title': 'American ...",5.0,"{'latitude': 40.73399149999999, 'longitude': -...","[pickup, delivery]","{'address1': '523 Hudson St', 'address2': '', ...",+16465843608,(646) 584-3608,3785.640809,NaN
999,drpANjzRLOaj9qOFiKo49Q,coffee-break-brooklyn,Coffee Break,https://s3-media3.fl.yelpcdn.com/bphoto/vVGI0c...,False,https://www.yelp.com/biz/coffee-break-brooklyn...,4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.6948, 'longitude': -73.95592}","[pickup, delivery]","{'address1': '683 Myrtle Ave', 'address2': Non...",+17182540044,(718) 254-0044,2384.788543,NaN


In [66]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

257

In [67]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [68]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_salad.csv.gz', compression='gzip',index=False)